<a href="https://colab.research.google.com/github/7Blessings7/Final-Project-CMS204/blob/main/Pair_17_Code_Espinosa_Emil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict


class Graph:
    def __init__(self, vertices=None, edges=None):
        if edges is None:
            edges = []
        self.vertices = [] if vertices is None else vertices
        self.V = len(self.vertices)
        self.graph = defaultdict(list)
        self.ids = {}
        self.keys = {}
        self.sc_components = {}
        self.process_id()
        self.add_edges(edges)

    # will process the generation of id of each vertices, the index as id
    def process_id(self):
        for i in range(self.V):
            self.ids[self.vertices[i]] = i
            self.keys[i] = self.vertices[i]

    # will return the id of the vertex
    def id(self, code):
        return self.ids[code]

    # will return the vertex name/code of the id
    def key(self, airport_id):
        return self.keys[airport_id]

    # will add edge to the graph
    def add_edge(self, source, destination):
        if source not in self.vertices:
            raise Exception("Source " + source + " not on the list of vertices")
        if destination not in self.vertices:
            raise Exception("Destination " + destination + " not on the list of vertices")
            
        src_id = self.id(source)
        dest_id = self.id(destination)
        self.graph[src_id].append(dest_id)

    # will add multiple edges to the graph, passing a list
    def add_edges(self, edges):
        for [source, destination] in edges:
            self.add_edge(source, destination)

    # A function used by DFS
    def dfs_util(self, v, visited, scc_component):
        # Mark the current node as visited and print it
        visited[v] = True
        scc_component.append(v)
        # print(self.key(v), end=' ')
        # Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if not visited[i]:
                self.dfs_util(i, visited, scc_component)
        return scc_component

    def fill_order(self, v, visited, stack):
        # Mark the current node as visited
        visited[v] = True
        # Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if not visited[i]:
                self.fill_order(i, visited, stack)
        stack = stack.append(v)

    # Function that returns reverse (or transpose) of this graph
    def get_transpose(self):
        _g = Graph(self.vertices)

        # Recur for all the vertices adjacent to this vertex
        for i in self.graph:
            for j in self.graph[i]:
                _g.add_edge(self.key(j), self.key(i))
        return _g

    # The main function that finds and prints all strongly
    # connected components
    def scc_transform(self):
        stack = []
        # Mark all the vertices as not visited (For first DFS)
        visited = [False] * self.V
        # Fill vertices in stack according to their finishing
        # times
        for i in range(self.V):
            if not visited[i]:
                self.fill_order(i, visited, stack)

        # Create a reversed graph
        gr = self.get_transpose()

        # Mark all the vertices as not visited (For second DFS)
        visited = [False] * self.V

        # Now process all vertices in order defined by Stack
        while stack:
            i = stack.pop()
            if not visited[i]:
                scc = gr.dfs_util(i, visited, [])
                self.sc_components[scc[0]] = scc

    def determine_additional_required_route(self, starting_point):
        if starting_airport not in self.vertices:
            self.ids[starting_point] = self.V
            self.V += 1
            
        answers = []
        self.scc_transform()
        
        for rep, components in self.sc_components.items():
            has_inbound = False
            for src, dest in self.graph.items():
                if rep in dest and src not in components:
                    has_inbound = True
                    break
            if not has_inbound and self.id(starting_point) not in components:
                answers.append(rep)

        if len(answers) == 0:
            print("Additional routes is not needed as travelers can now to go every other destination")
        else:
            print("The new routes that should be enabled to reach every other airport from a fixed starting airport"
                  " is/are:")
            for ans in answers:
                additional_route = [self.key(component) for component in self.sc_components[ans]]

                if len(additional_route) > 1:
                    print(starting_point + " to any of: [ " + ", ".join(additional_route) + " ]")
                else:
                    print(starting_point + " to " + additional_route[0])


airports = ["ATL", "PEK", "PKX", "LHR", "HND", "ORD", "LAX", "CDG", "DFW", "CGK", "MNL", "DXB", "FRA", "HKG", "DEN",
            "BKK", "SIN", "AMS"]

routes = [
    ["HND", "FRA"],
    ["FRA", "ATL"],
    ["ATL", "MNL"],
    ["BKK", "PEK"],
    ["PEK", "BKK"],
    ["PEK", "AMS"],
    ["PKX", "LHR"],
    ["PKX", "PEK"],
    ["SIN", "PKX"],
    ["ORD", "CDG"],
    ["CDG", "DFW"],
    ["CDG", "CGK"],
    ["DFW", "CGK"],
    ["CGK", "MNL"],
    ["LAX", "DXB"],
    ["DXB", "DEN"],
    ["DEN", "HKG"],
    ["DEN", "HND"],
    ["HKG", "LAX"]
]
starting_airport = "MNL"
g = Graph(airports, routes)
# g.graph
g.determine_additional_required_route(starting_airport)
# g.sc_components

The new routes that should be enabled to reach every other airport from a fixed starting airport is/are:
MNL to SIN
MNL to any of: [ LAX, HKG, DEN, DXB ]
MNL to ORD
